In [7]:
# @title 1. Install Dependencies
!pip install -q torch transformers datasets accelerate huggingface_hub python-dotenv matplotlib liger-kernel
!pip install -q git+https://github.com/KellerJordan/Muon # Install Muon Optimizer

Loss: 4.1771 | LR: 4.40e-06:   0%|          | 11/50000 [24:59<1893:09:47, 136.34s/it]
  0%|          | 0/50000 [29:49<?, ?it/s]


  Preparing metadata (setup.py) ... done


In [ ]:

import os
import math
import time
import glob
import re
from dataclasses import dataclass
from contextlib import nullcontext

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.utils.data import IterableDataset, DataLoader
from transformers import AutoTokenizer
from datasets import load_dataset
from huggingface_hub import HfApi, create_repo
import tqdm
import matplotlib.pyplot as plt

# Try to import Muon and Liger, fallback if missing
try:
    from muon import MuonWithAuxAdam
    HAS_MUON = True
    print("✅ Muon Optimizer enabled.")
except ImportError:
    HAS_MUON = False
    print("⚠️ Muon not found, using AdamW.")

try:
    from liger_kernel.transformers import LigerFusedLinearCrossEntropyLoss
    HAS_LIGER = True
    print("✅ Liger Kernel enabled.")
except ImportError:
    HAS_LIGER = False
    print("⚠️ Liger not found, using standard CrossEntropy.")

# ==========================================
# 1. CONFIGURATION
# ==========================================
@dataclass
class ModelArgs:
    # --- Architecture ---
    dim: int = 384              
    n_layers: int = 6
    n_heads: int = 6               
    n_kv_heads: int = 2
    
    # --- MoE Config ---
    num_experts: int = 8            
    num_shared_experts: int = 1
    top_k: int = 2
    expert_hidden_dim: int = 1024
    
    # --- MLA Config ---
    kv_lora_rank: int = 64         
    q_lora_rank: int = 256   
    rope_theta: float = 10000.0
    norm_eps: float = 1e-6
    
    # --- Training ---
    vocab_size: int = 50257         # GPT-2 Vocab for TinyStories
    max_seq_len: int = 512          # Short context
    batch_size: int = 32       # Fits 14GB VRAM
    gradient_accumulation_steps: int = 2
    
    # --- Optimization ---
    lr_decay_iters: int = 50000     
    warmup_iters: int = 1000
    max_lr: float = 6e-4            
    min_lr: float = 6e-5
    weight_decay_optim: float = 0.1
    clip: float = 1.0
    total_iters: int = 50000
    eval_iters: int = 200
    save_checkpoint_iter: int = 1000
    dropout: float = 0.0
    aux_loss_coef: float = 0.01
    
    # --- System ---
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    use_liger: bool = True 
    dataset: str = "roneneldan/TinyStories" 
    checkpoint_dir: str = "checkpoints_tinystories"
    #hf_token: str = os.getenv("HF_TOKEN")
    hf_token: str = ""
    hf_repo_id: str = "FusionCorp/DeepSeek-V3-TinyStories"
    gradient_checkpointing: bool = True 



✅ Muon Optimizer enabled.
✅ Liger Kernel enabled.


In [ ]:
# ==========================================
# 2. MODEL DEFINITION
# ==========================================
class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        var = torch.mean(x ** 2, dim=-1, keepdim=True)
        x = x * torch.rsqrt(var + self.eps)
        return x * self.weight

def apply_rotary_emb(xq, xk, freq_cis):
    xq_out = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
    xk_out = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))
    freq_cis = freq_cis[:xq.shape[1]]

    freq_cis = freq_cis.view(1, xq.shape[1], 1, -1)
    
    xq_out = torch.view_as_real(xq_out * freq_cis).flatten(3)
    xk_out = torch.view_as_real(xk_out * freq_cis).flatten(3)
    return xq_out.type_as(xq), xk_out.type_as(xk)

class MLA(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.dim = args.dim
        self.n_heads = args.n_heads
        self.head_dim = args.dim // args.n_heads
        self.kv_lora_rank = args.kv_lora_rank
        
        self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.w_kv_down = nn.Linear(args.dim, args.kv_lora_rank, bias=False)
        self.w_kv_up = nn.Linear(args.kv_lora_rank, 2 * (args.n_heads * self.head_dim), bias=False)
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)
        
        self.q_norm = RMSNorm(self.head_dim)
        self.k_norm = RMSNorm(self.head_dim)
        self.register_buffer("freqs_cis", self.precompute_freqs_cis(args.dim // args.n_heads, args.max_seq_len))

    def precompute_freqs_cis(self, dim: int, end: int, theta: float = 10000.0):
        freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
        t = torch.arange(end, device=freqs.device)
        freqs = torch.outer(t, freqs).float()
        freqs_cis = torch.polar(torch.ones_like(freqs), freqs)
        return freqs_cis

    def forward(self, x):
        B, T, C = x.shape
        xq = self.wq(x).view(B, T, self.n_heads, self.head_dim)
        latent_kv = self.w_kv_down(x)
        kv = self.w_kv_up(latent_kv).view(B, T, 2, self.n_heads, self.head_dim)
        xk, xv = kv.unbind(2)
        
        xq, xk = self.q_norm(xq), self.k_norm(xk)
        xq, xk = apply_rotary_emb(xq, xk, self.freqs_cis)
        
        out = F.scaled_dot_product_attention(
            xq.transpose(1, 2), xk.transpose(1, 2), xv.transpose(1, 2), is_causal=True
        )
        return self.wo(out.transpose(1, 2).contiguous().view(B, T, C))

class DeepSeekMoE(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.num_experts = args.num_experts
        self.top_k = args.top_k
        self.num_shared = args.num_shared_experts
        self.gate = nn.Linear(args.dim, args.num_experts, bias=False)
        
        self.shared_experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(args.dim, args.expert_hidden_dim, bias=False), 
                nn.SiLU(), 
                nn.Linear(args.expert_hidden_dim, args.dim, bias=False)
            ) for _ in range(self.num_shared)
        ])
        self.routed_experts = nn.ModuleList([
             nn.Sequential(
                nn.Linear(args.dim, args.expert_hidden_dim, bias=False), 
                nn.SiLU(), 
                nn.Linear(args.expert_hidden_dim, args.dim, bias=False)
            ) for _ in range(self.num_experts)
        ])

    def forward(self, x):
        original_shape = x.shape
        x_flat = x.view(-1, original_shape[-1])
        
        # 1. Compute Shared Experts (always active)
        shared_out = sum(expert(x_flat) for expert in self.shared_experts)
        
        # 2. Gating and Top-K
        logits = self.gate(x_flat)
        probs = F.softmax(logits, dim=-1)
        top_k_weights, top_k_indices = torch.topk(probs, self.top_k, dim=-1)
        
        # Normalize weights so they sum to 1
        top_k_weights = top_k_weights / top_k_weights.sum(dim=-1, keepdim=True)
        
        # Aux Loss for load balancing
        aux_loss = (probs.mean(dim=0) * logits.mean(dim=0)).sum() * self.num_experts
        
        final_out = torch.zeros_like(x_flat)
        
        # 3. Process each Expert
        for i in range(self.num_experts):
            # Check which tokens and which k-slots (0 or 1) use expert i
            # mask shape: [num_tokens, top_k]
            mask = (top_k_indices == i) #sets mask to true whenever we reach the wanted expert
            if not mask.any():
                continue
            
            # We process this expert once for all tokens that need it.
            # To handle multiple slots (if a token picks the same expert twice) we iterate through the k-slots.
            for k in range(self.top_k):
                k_mask = (top_k_indices[:, k] == i)
                if k_mask.any():
                    # Run expert on the relevant tokens and multiply by routing weight
                    expert_input = x_flat[k_mask]
                    expert_output = self.routed_experts[i](expert_input)
                    
                    # Add to final output
                    final_out[k_mask] += expert_output * top_k_weights[k_mask, k].unsqueeze(-1)

        return (shared_out + final_out).view(*original_shape), aux_loss

class Block(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.attn_norm = RMSNorm(args.dim)
        self.attn = MLA(args)
        self.ffn_norm = RMSNorm(args.dim)
        self.moe = DeepSeekMoE(args)
        self.dropout = nn.Dropout(args.dropout)

    def forward(self, x):
        h = x + self.dropout(self.attn(self.attn_norm(x)))
        moe_out, aux_loss = self.moe(self.ffn_norm(h))
        return h + self.dropout(moe_out), aux_loss

class DeepSeekV3(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.embedding = nn.Embedding(args.vocab_size, args.dim)
        self.layers = nn.ModuleList([Block(args) for _ in range(args.n_layers)])
        self.norm = RMSNorm(args.dim)
        self.linear_layer = nn.Linear(args.dim, args.vocab_size, bias=False)
        self.embedding.weight = self.linear_layer.weight # Weight tying
        self.last_aux_loss = 0.0
        
        if args.use_liger and HAS_LIGER:
            self.le_loss = LigerFusedLinearCrossEntropyLoss()
            
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, input_ids):
        x = self.embedding(input_ids)
        #total_aux_loss = 0.0

        total_aux_loss = torch.tensor(0.0, device=x.device, dtype=x.dtype)

        for layer in self.layers:
            if self.training and self.args.gradient_checkpointing:
                x, aux_loss = torch.utils.checkpoint.checkpoint(layer, x, use_reentrant=True)
            else:
                x, aux_loss = layer(x)

            # trying x = x + 1 instead                
            total_aux_loss = total_aux_loss + aux_loss
        
        x = self.norm(x)
        self.last_aux_loss = total_aux_loss
        if self.args.use_liger and self.training: return x
        return self.linear_layer(x)

# ==========================================
# 3. DATA PIPELINE
# ==========================================
def initialize_tokenizer(hf_token=None):
    try:
        tokenizer = AutoTokenizer.from_pretrained("gpt2", token=hf_token)
    except:
        tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

class TinyStoriesStreamDataset(IterableDataset):
    def __init__(self, split, tokenizer, seq_len, dataset_name, hf_token=None):
        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.dataset = load_dataset(dataset_name, split="train" if split == "train" else "validation", streaming=True, token=hf_token)

    def __iter__(self):
        iterator = iter(self.dataset)
        buffer = []
        while True:
            try:
                text = next(iterator)['text']
                tokens = self.tokenizer.encode(
                text,
                add_special_tokens=True,
                max_length = 1e6,
                truncation = False ) # to get rid of the HF warning

                buffer.extend(tokens)
                while len(buffer) >= self.seq_len + 1:
                    chunk = buffer[:self.seq_len + 1]
                    buffer = buffer[self.seq_len + 1:]
                    yield {'input_ids': torch.tensor(chunk[:-1]), 'labels': torch.tensor(chunk[1:])}
            except StopIteration:
                iterator = iter(self.dataset) # Infinite loop



In [13]:
# ==========================================
# 4. TRAINING UTILS
# ==========================================
def get_lr(it, args):
    if it < args.warmup_iters: return args.max_lr * (it + 1) / args.warmup_iters
    if it > args.lr_decay_iters: return args.min_lr
    decay_ratio = (it - args.warmup_iters) / (args.lr_decay_iters - args.warmup_iters)
    return args.min_lr + 0.5 * (args.max_lr - args.min_lr) * (1.0 + math.cos(math.pi * decay_ratio))

def find_latest_checkpoint(checkpoint_dir):
    if not os.path.exists(checkpoint_dir): os.makedirs(checkpoint_dir, exist_ok=True); return None, 0
    files = glob.glob(os.path.join(checkpoint_dir, "checkpoint_*.pt"))
    if not files: return None, 0
    latest = max(files, key=lambda f: int(re.search(r'checkpoint_(\d+).pt', f).group(1)))
    return latest, int(re.search(r'checkpoint_(\d+).pt', latest).group(1))

# ==========================================
# 5. MAIN TRAINING LOOP
# ==========================================
def train():

    torch.cuda.empty_cache()
    import gc
    gc.collect()


    # Necessary for Muon to work
    if not dist.is_initialized():
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = '12355'
        dist.init_process_group(backend='nccl', rank=0, world_size=1)


    args = ModelArgs()
    tokenizer = initialize_tokenizer(args.hf_token)
    print(f"🚀 Device: {args.device} | Vocab: {len(tokenizer)}")

    model = DeepSeekV3(args).to(args.device)
    
    # Optimizer Groups
    hidden_params, other_params = [], []
    for n, p in model.named_parameters():
        if p.requires_grad:
            (hidden_params if p.ndim >= 2 and "norm" not in n and "embedding" not in n else other_params).append(p)

    if HAS_MUON:
        optimizer = MuonWithAuxAdam([
            {'params': hidden_params, 'use_muon': True, 'lr': 0.02, 'weight_decay': 0.01},
            {'params': other_params, 'use_muon': False, 'lr': args.max_lr, 'weight_decay': args.weight_decay_optim}
        ])
    else:
        optimizer = torch.optim.AdamW(model.parameters(), lr=args.max_lr, weight_decay=args.weight_decay_optim)

    # Resume
    ckpt_path, start_step = find_latest_checkpoint(args.checkpoint_dir)
    if ckpt_path:
        print(f"⏩ Resuming from {start_step}")
        ckpt = torch.load(ckpt_path, map_location=args.device)
        model.load_state_dict(ckpt['model'])
        optimizer.load_state_dict(ckpt['optimizer'])

    # Data
    ds = TinyStoriesStreamDataset('train', tokenizer, args.max_seq_len, args.dataset, args.hf_token)
    loader = DataLoader(ds, batch_size=args.batch_size, num_workers=2, pin_memory=True)
    iterator = iter(loader)

    # Training Loop
    model.train()
    pbar = tqdm.tqdm(range(start_step, args.total_iters), initial=start_step)
    accum_loss = 0
    
    print("🚀 Initialization complete. Starting training loop...")

    for step in range(start_step, args.total_iters):
        # LR Schedule
        lr = get_lr(step, args)
        for g in optimizer.param_groups: 
            if not g.get('use_muon', False): g['lr'] = lr
        
        optimizer.zero_grad()
        for micro in range(args.gradient_accumulation_steps):
            try: batch = next(iterator)
            except: iterator = iter(loader); batch = next(iterator)
            
            idx, targets = batch['input_ids'].to(args.device), batch['labels'].to(args.device)
            
            with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
                out = model(idx)
                if args.use_liger and HAS_LIGER:
                    loss = model.le_loss(model.linear_layer.weight, out.view(-1, args.dim), targets.view(-1))
                else:
                    loss = F.cross_entropy(out.view(-1, args.vocab_size), targets.view(-1))
                
                loss = (loss + args.aux_loss_coef * model.last_aux_loss) / args.gradient_accumulation_steps
            loss.backward()
            accum_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()

        if step % 10 == 0:
            print(f"Step {step} | Loss: {accum_loss:.4f} | LR: {lr:.2e}")
        
        pbar.update(1)
        pbar.set_description(f"Loss: {accum_loss:.4f} | LR: {lr:.2e}")
        accum_loss = 0

        if step % args.save_checkpoint_iter == 0 and step > start_step:
            path = os.path.join(args.checkpoint_dir, f"checkpoint_{step}.pt")
            torch.save({'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'args': args}, path)
            print(f"\n💾 Saved {path}")

# Run it
train()

  0%|          | 0/50000 [24:31<?, ?it/s]


🚀 Device: cuda | Vocab: 50257


🚀 Initialization complete. Starting training loop...


Step 0 | Loss: 11.0061 | LR: 4.00e-07


Step 10 | Loss: 4.3378 | LR: 4.40e-06


Token indices sequence length is longer than the specified maximum sequence length for this model (1050 > 1024). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 